# Generating genomic feature tracks

In [38]:
!reference="/project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/Cvirginica_genome/CV_working_genomic.gff"

In [39]:
pwd

'/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/genomic_feature_tracks'

In [28]:
genome="/project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/Cvirginica_genome/GCF_C_virginica-3.0_genomic.fai"
!cpgList="/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_methyl_assembly/assembly_pipeline_files/genomic_bed_files/C_virginica-3.0_CG-motif.bed"
gff="/project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/Cvirginica_genome/CV_working_genomic.gff"
chromLengths="/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/genomic_feature_tracks/Venkataraman_files/2018-06-15-bedtools-Chromosome-Lengths.txt"

In [10]:
!cut -f1 $chromLengths > Chromosome_Names.txt

!chromNames="/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/genomic_feature_tracks/Chromosome_Names.txt"

^C


## Genes

In [21]:
# find features with type == gene
!grep "Gnomon	gene" $gff > CV_gene.gff3

^C


38262 genes in CV genome

In [ ]:
# sort for downstream use
sortBed -faidx $chromNames -i CV_gene.gff3 > CV_gene_sorted.gff3

In [40]:
!geneList="CV_gene_sorted.gff3"

In [ ]:
# convert gff file to BED file
!awk '{print $1"\t"$4"\t"$5}' $geneList \
> CV_gene_sorted.bed

## Exons

In [ ]:
# find features with type == exon
!grep "Gnomon	exon" $gff > CV_exon.gff3

731279 exons in CV genome

In [ ]:
sortBed -faidx $chromNames -i CV_exon.gff3 > CV_exon_sorted.gff3

In [29]:
!exonList="CV_exon_sorted.gff3"

In [ ]:
# convert gff file to BED file
!awk '{print $1"\t"$4"\t"$5}' $exonList \
> CV_exon_sorted.bed

## CDS

In [ ]:
# find features with type == exon
!grep "Gnomon	CDS" $gff > CV_CDS.gff3

645355 CDS in CV genome

In [ ]:
sortBed -faidx $chromNames -i CV_CDS.gff3 > CV_CDS_sorted.gff3

In [30]:
!cdsList="CV_CDS_sorted.gff3"

In [ ]:
# convert gff file to BED file
!awk '{print $1"\t"$4"\t"$5}' $cdsList \
> CV_CDS_sorted.bed

## mRNA

In [ ]:
# find features with type == exon
!grep "Gnomon	mRNA" $gff > CV_mRNA.gff3

60201 mRNA in CV genome

In [31]:
!mRNAList="CV_mRNA.gff3"

In [ ]:
# convert gff file to BED file
!awk '{print $1"\t"$4"\t"$5}' $mRNAList \
> CV_mRNA.bed

## Intergenic regions
regions that aren't genes

`complementBed` to find regions that aren't genes, and `subtractBed` to remove exons and create this

In [ ]:
complementBed \
-i $geneList -sorted \
-g $chromLengths \
| subtractBed \
-a - \
-b $exonList \
> CV_intergenic.gff3

In [32]:
!intergenicList="CV_intergenic.gff3"

39467 intergenic regions in CV genome

In [ ]:
awk '{print $1"\t"$2"\t"$3}' $intergenicList \
> CV_intergenic.bed

## non-coding sequences

In [ ]:
complementBed -i $exonList -g $chromLengths > CV_noncoding.gff3

In [33]:
!nonCDS="CV_noncoding.gff3"

336677 non-coding sequences in CV genome

In [ ]:
awk '{print $1"\t"$2"\t"$3}' $nonCDS \
> CV_noncoding.bed

### introns

introns are the space between exons within a gene - so to pull this out, I have to look within a gene (LOC number), subtract the end of exon 1 from the start of exon 2


can create GFF file of non-coding regions based on the original GFF file - then introns, by definition, are the intersections of non-coding regions and genes


following pipeline from [Venkataraman et al 2020](https://www.frontiersin.org/journals/marine-science/articles/10.3389/fmars.2020.00225/full#h7)

In [ ]:
# run in command line
!intersectBed \
-a $nonCDS \
-b $geneList -sorted \
> CV_intron.gff3

In [34]:
!intronList="CV_intron.gff3"

311168 introns in CV genome

In [ ]:
awk '{print $1"\t"$2"\t"$3}' $intronList \
> CV_intron.bed

### exon UTRs

In [ ]:
subtractBed -a $exonList -b $cdsList -sorted -g $chromLengths > CV_exonUTR.gff3

In [35]:
!exonUTR="CV_exonUTR.gff3"

9473 untranslated regions of exons in CV genome

In [ ]:
awk '{print $1"\t"$4"\t"$5}' $exonUTR \
> CV_exonUTR.bed

### putative promoters
1KB upstream of transcription start site (TSS)

can use `bedtools flank` to find flanking regions 1000bp upstream and downstream of mRNA - then can filter rows to only grab the upstream flank (odd rows) on the + strand which would be our putative promoters



In [ ]:
flankBed -i $mRNAList -g $chromLengths -b 1000 > mRNA_1000bp_flanks.bed

In [ ]:
flanks='mRNA_1000bp_flanks.bed'

In [ ]:
wc -l mRNA_1000bp_flanks.bed

120400

if row number is odd = upstream flank

if row number is even = downstream flank

In [ ]:
awk '{ if (NR%2) print > "mRNA_upstream_flanks.bed"; \
else print > "mRNA_downstream_flanks.bed" }' \
$flanks

In [ ]:
upstream="mRNA_upstream_flanks.bed"
downstream="mRNA_downstream_flanks.bed"

**upstream flanks**

60200 upstream flanks

**downstream flanks**
60200 downstream flanks



upstream flanks should only include + strands, downstream should only include - strands

In [ ]:
grep ".	+	." $upstream \
> mRNA_upstream_forward.bed

wc -l mRNA_upstream_forward.bed

30218

In [ ]:
grep ".	-	." $downstream \
> mRNA_downstream_reverse.bed

wc -l mRNA_downstream_reverse.bed

29982

now combine upstream + and downstream - flanks to create promoter track

In [ ]:
cat \
mRNA_upstream_forward.bed \
mRNA_downstream_reverse.bed \
> mRNA_promoter_track.bed

In [36]:
!promoterTrack="mRNA_promoter_track.bed"

60200

# Overlaps with CpG dinucleotides

In [57]:
!conda run -n bedtools bedtools intersect -u -a $cpgList -b /project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/genomic_feature_tracks/CV_gene_sorted.bed | wc -l
#echo "total CpG dinucleotides in genes"

Error: Unable to open file -b. Exiting.

ERROR conda.cli.main_run:execute(125): `conda run bedtools intersect -u -a -b /project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_MethylRAD_analysis_2018/analysis/genomic_feature_tracks/CV_gene_sorted.bed` failed. (See above for error)
0
